In [1]:
#include <iostream>
#include <list>
using namespace std;

# 7.1 - Trees

A tree $T$ can be defined as a set of nodes which store elements that exist in a **parent-child relationship.** 

Properties of a tree $T$:
- If $T$ is nonempty, then it has a node called its **root** which has no parent
- Each *other* node of $T$ has a **unique** parent $w$, and every node with parent $w$ is a child of $w$

Using these properties, trees can be defined **recursively**:
- A tree $T$ is either empty, or it contains a root $r$ whose children are the roots of a set of (sub)trees.

Tree terminology regarding relationships between nodes and substructures:
- Siblings: two or more nodes which have the same parent
- Internal node: any node that has at least one child
- Leaf/External node: any node which has no children
- Ancestor: A node $v$ is an ancestor of a node $w$ if $v = w$ or if $v$ is an ancestor of a parent of $w$
- Subtree of $T$ rooted at $v$: describes the tree which consists of node $v$ and all its descendents in the tree $T$
- Edge: a pair of parent-child nodes, ex. $(u,v)$
- Path: a sequence of parent-child nodes, or a sequence of nodes in which any two consecutive nodes form an edge
- Ordered tree: describes a tree in which the left-to-right arrangement of siblings corresponds to a relationship that is relevant to the usage of the tree

**The Tree API**
    
Recall that the *position* data type abstracts the notion of the relative position or place of an element within a data structure. Because nodes are internal aspects of the implementation of trees, it is appropriate to encapsulate the nodes using position objects. Practically, this means that the arguments to Tree methods are positions, not nodes. However, we can still conceptually think of the methods as acting on nodes.

The Position interface consists of five methods:

In [2]:
template <typename E>
class Position<E> {
public:
  E& operator*();                     // get the node's element
  Position parent() const;            // get parent
  PositionList children() const;      // get a list of the node's children
  bool isRoot() const;                // returns true if root, false otherwise
  bool isExternal() const;            // returns true if external, false otherwise
};

input_line_9:2:7: error: explicit specialization of non-template class 'Position'
class Position<E> {
      ^       ~~~
input_line_9:6:3: error: unknown type name 'PositionList'; did you mean 'Position'?
  PositionList children() const;      // get a list of the node's children
  ^~~~~~~~~~~~
  Position
input_line_9:2:7: note: 'Position' declared here
class Position<E> {
      ^


Interpreter Error: 

The Tree interface consists of four methods:

In [ ]:
template <typename E>
class Tree<E> {
public:
  class Position;
  class PositionList;                      // give public access to Position and PositionList; this is 
                                           // how other objects will use the Tree
  
  int size() const;       
  bool empty() const;
  Position root() const;                   // get the root of the tree
  PositionList positions() const;          // get a list of positions referring to each node in the
                                           // tree

Trees can be implemented using a **linked structure** in which each node of the tree is represented by a position object that has three fields:
- Pointer to the node's parent (NULL, if root)
- Reference to the node's element
- Pointer to a container holding the node's children (a list, in the above interface)

Then the running times of the functions are:

| Operation | Running time   |
|------|------|
|   isRoot, isExternal  | O(1) |
| parent | O(1) |
| children(p) | O($c_p$)|
| size, empty | O(1) |
| root | O(1) |
| positions | O($n$) |

Where $c_p$ is the number of children of the node referenced by $p$.

# 7.2 - Tree Traversal Algorithms

Trees can be traversed using the tree ADT methods described above. We define some useful properties of trees in their traversal.

**Depth**

The depth of a node $p$ in a tree $T$ is defined as the number of ancestors of $p$, excluding itself. Alternatively, the depth of node $p$ can be defined recursively as:
- Zero, if $p$ is a root
- One plus the depth of the parent of $p$

The recursive definition is probably the easiest to implement in C++.

In [3]:
int depth(const Tree& tree, const Position& node) {
    if (node.isRoot()) return 0;
    return 1 + depth(tree, node.parent());
}

input_line_10:1:17: error: unknown type name 'Tree'
int depth(const Tree& tree, const Position& node) {
                ^
input_line_10:1:35: error: unknown type name 'Position'
int depth(const Tree& tree, const Position& node) {
                                  ^


Interpreter Error: 

The running time of depth(Tree, node) is $O$(node depth) because it must recursively call itself until the input node becomes the root. The worst-case running time is $O(n)$, where $n$ is the total number of nodes in the tree, if some nodes have depth $n$.

**Height** 

Whereas depth is a measure of the downwards distance of a node from the root, height is a measure of the upwards distance of a node from the lowest connected node. 

The height of a node $p$ in a tree $T$ can be defined recursively as:
- Zero, if $p$ is an external node
- One plus the maximum height among the children of $p$

The height of a **tree** $T$ is the height of the root node of $T$. The height of a tree can also be viewed as the **maximum depth** among all external nodes. Using the recursive definition to implement a C++ function is more efficient than using the maximum depth definition. 

In [2]:
int height(const Tree& tree, const Position& node) {
    if (node.isExternal()) return 0;
    PositionList children = node.children();
    int max_height_children = 0;
    for (Iterator node = children.begin(); node != children.end(); ++node) {
        max_height_children = max( max_height_children, height(tree, node) )
    }
    return 1 + max_height_children;
}

input_line_8:1:18: error: unknown type name 'Tree'
int height(const Tree& tree, const Position& node) {
                 ^
input_line_8:1:36: error: unknown type name 'Position'
int height(const Tree& tree, const Position& node) {
                                   ^
input_line_8:3:5: error: unknown type name 'PositionList'
    PositionList children = node.children();
    ^
input_line_8:5:10: error: unknown type name 'Iterator'
    for (Iterator node = children.begin(); node != children.end(); ++node) {
         ^


Interpreter Error: 

The running time of the recursive height() function is $O(n)$, by the following analysis:
- If height() is called on the root of $T$, it will eventually be called on every node of $T$ because each recursive call iterates through all the children of the input node
- Therefore, the running time of the function is the sum of the time the function spends on each individual node
- At each node, height() calls node.children() which takes $O(c_p)$ time to return a PositionList of the node's children, where $c_p$ is the number of children of node $p$
- The for loop also takes $O(c_p)$ time to complete
    - Each iteration takes $O(1)$ time to do the assignment plus the time for the recursive call to height() on the child node
- Therefore, each call takes $O(1+c_p)$ time and the whole function takes $O(\sum_p (1+c_p))$ time to process all the nodes in the tree
- If a tree has $n$ nodes, then $\sum_p c_p = n-1$ because every node except for the root is a child node of some parent node
- Therefore, height() runs in $O(n)$ time.


**Preorder Traversal**

A *traversal* refers to a systematic way of visiting all the nodes in a tree $T$. In a preorder traversal of $T$, each parent node (including the root) is visited before their children. If the tree is ordered, the the children should be visited in order. Then the tree is processed "top-down", in the sense that 

In [3]:
void preorderTraversal(const Tree& tree, const Position& node) {
    // Process the node with code here
    PositionList children = node.children();
    for (Iterator node = children.begin(); node != children.end(); ++node) {
        preorderTraversal(tree, node);
    }
}

input_line_9:1:30: error: unknown type name 'Tree'
void preorderTraversal(const Tree& tree, const Position& node) {
                             ^
input_line_9:1:48: error: unknown type name 'Position'
void preorderTraversal(const Tree& tree, const Position& node) {
                                               ^
input_line_9:3:5: error: unknown type name 'PositionList'
    PositionList children = node.children();
    ^
input_line_9:4:10: error: unknown type name 'Iterator'
    for (Iterator node = children.begin(); node != children.end(); ++node) {
         ^


Interpreter Error: 

Preorder traversals should be used when parent nodes must be processed before their children (ex. the sections of a white paper). In general, a preorder traversal is an efficient method of visiting all the nodes in a tree, which occurs in $O(n)$ time. The analysis is as follows:
- Suppose processing a node takes $O(1)$ time
- At each node, the traversal calls node.children() which takes $O(c_p)$ time to return $c_p$ children
- Therefore, the entirety of the non-recursive part of the algorithm takes $O(1+c_p)$ time
- If preorderTraversal() is called on the root node, all nodes will eventually be visited, so the whole algorithm takes $\sum_p (1+c_p)$ time
- Since $\sum_p c_p = n-1$ the algorithm takes $O(n)$ time

**Postorder Traversal**

In a postorder traversal of a tree $T$, children nodes are processed before parent nodes. Then the tree is processed "bottom-up", in the sense that the traversal will visit a given node $p$ after it has visited all the other nodes in the subtree rooted at $p$.

In [4]:
void postorderTraversal(const Tree& tree, const Position& node) {
    PositionList children = node.children();
    for (Iterator node = children.begin(); node != children.end(); ++node) {
        preorderTraversal(tree, node);
    }
    // Process the node with code here
}

input_line_10:1:31: error: unknown type name 'Tree'
void postorderTraversal(const Tree& tree, const Position& node) {
                              ^
input_line_10:1:49: error: unknown type name 'Position'
void postorderTraversal(const Tree& tree, const Position& node) {
                                                ^
input_line_10:2:5: error: unknown type name 'PositionList'
    PositionList children = node.children();
    ^
input_line_10:3:10: error: unknown type name 'Iterator'
    for (Iterator node = children.begin(); node != children.end(); ++node) {
         ^


Interpreter Error: 

Which has time complexity $O(n)$ for the same reasons as the preorder traversal. 

Postorder traversals should be used when we want to compute a property of a node $p$ that requires the same property to have been computed for all children of $p$. An example of this is determining the disk space used by a directory that is organzied in a tree structure; we need to first determine the sizes of the subdirectories and then sum them to determine the size of the main directory.

Remark: both preorder and postorder traversals implicitly use a stack through their recursive calls. It is possible to implement these traversals without recursion by directly using a stack.

# 7.3 - Binary Trees

A binary tree is an ordered tree in which every node has **at most** two children. The children are referred to as *left* and *right* and a left child precedes a right children in the ordering of the node's children.

The **left subtree** and **right subtree** refer to the subtrees rooted at the left and right child node, respectively. Thus, any binary tree can be recursively defined as a root and two binary trees (left and right).

A binary tree is called **proper** or **full** if each node has either zero or two children. Thus, every *internal* node will have exactly two children. Otherwise, the tree is called **improper**. 

Examples: both decision trees and arithmetic expressions can be represented by proper binary trees.

As before, we can use positions to encapsulate binary tree nodes in an implementation.

In [5]:
template <T>
class BTPosition<T> {
public:
  BTPosition left() const;            // get left child
  BTPosition right() const;           // get right child
  BTPosition parent() const;
  bool isRoot() const;
  bool isExternal() const;
};

input_line_11:1:11: error: unknown type name 'T'
template <T>
          ^
input_line_11:2:18: error: use of undeclared identifier 'T'
class BTPosition<T> {
                 ^
input_line_11:2:7: error: explicit specialization of non-template class 'BTPosition'
class BTPosition<T> {
      ^


Interpreter Error: 

Then the binary tree implementation is essentially the same as any other tree.

In [6]:
template <T>
class BinaryTree<T> {
public:
  class BTPosition;
  class PositionList;
  
  int size();
  int empty()
  BTPosition root();
  PositionList positions();
};

input_line_12:1:11: error: unknown type name 'T'
template <T>
          ^
input_line_12:2:18: error: use of undeclared identifier 'T'
class BinaryTree<T> {
                 ^
input_line_12:2:7: error: explicit specialization of non-template class 'BinaryTree'
class BinaryTree<T> {
      ^
input_line_12:8:14: error: expected ';' at end of declaration list
  int empty()
             ^
             ;


Interpreter Error: 

**Properties of Binary Trees**